In [29]:
class Feature:
    '''
    This is a class for features from geojson.
    '''
    def __init__(self, features):
        # set the MBR for polygon and multipolygon:
        min_x = float('inf') # start with something much higher than expected min
        min_y = float('inf')
        max_x = float('-inf') # start with something much lower than expected max
        max_y = float('-inf')
        
        mbrs = []
        mbrs.append([])# list for main polygon
        mbrs.append([])# list for holes

        '''
        No holes:
            { "type": "Polygon",
                "coordinates": [
                                [ [100.0, 0.0], [101.0, 0.0], [101.0, 1.0], [100.0, 1.0], [100.0, 0.0] ]
                               ]
            }
            
        With holes:
            { "type": "Polygon",
              "coordinates": [
                              [ [100.0, 0.0], [101.0, 0.0], [101.0, 1.0], [100.0, 1.0], [100.0, 0.0] ],
                              [ [100.2, 0.2], [100.8, 0.2], [100.8, 0.8], [100.2, 0.8], [100.2, 0.2] ]
                             ]
            }
        '''
            
        if features['geometry']['type'] == 'Polygon':
            # for the polygon without holes:
            for item in features['geometry']['coordinates'][0]:   
                if item[0] < min_x:
                    min_x = item[0]
                if item[0] > max_x:
                    max_x = item[0]
                if item[1] < min_y:
                    min_y = item[1]
                if item[1] > max_y:
                    max_y = item[1]
            mbrs[0].append([min_x,max_x,min_y,max_y])
            # for the polygon with holes:
            if len(features['geometry']['coordinates'])>1:
                for polygons in features['geometry']['coordinates'][1:]: 
                    for item in polygons:
                        if item[0] < min_x:
                            min_x = item[0]
                        if item[0] > max_x:
                            max_x = item[0]
                        if item[1] < min_y:
                            min_y = item[1]
                        if item[1] > max_y:
                            max_y = item[1]
                    mbrs[1].append([min_x,max_x,min_y,max_y])
                '''
                The result is [[[x1,x2,y1,y2]],[[x3,x4,y3,y4],[x5,x6,y5,y6],[x7,x8,y7,y8]]]
                mbrs[0]--> for the main polygon
                mbrs[1]--> for all of holes in polygon
                '''
                '''
                ------------------------------------------------------------------------------------------
                '''
                
                '''
                { "type": "MultiPolygon",
                  "coordinates": [
                                    [
                                     [[102.0, 2.0], [103.0, 2.0], [103.0, 3.0], [102.0, 3.0], [102.0, 2.0]]
                                    ],# island without hole
                                    
                                    [
                                     [[100.0, 0.0], [101.0, 0.0], [101.0, 1.0], [100.0, 1.0], [100.0, 0.0]],
                                     [[100.2, 0.2], [100.8, 0.2], [100.8, 0.8], [100.2, 0.8], [100.2, 0.2]]
                                    ] # island with one hole
                                 ]
                }
                '''           

        elif features['geometry']['type'] == 'MultiPolygon':
            for island in features['geometry']['coordinates']:
                # each island could be island or island with holes
                if len(island)==1: # without hole
                    for polygon in island:
                        # the first polygon is main polygon, and else polygons should be the holes
                        for item in polygon:   
                            if item[0] < min_x:
                                min_x = item[0]
                            if item[0] > max_x:
                                max_x = item[0]
                            if item[1] < min_y:
                                min_y = item[1]
                            if item[1] > max_y:
                                max_y = item[1]
                        mbrs[0].append([min_x,max_x,min_y,max_y])

                if len(island)>1:# with holes
                    for item in island[0]:   
                        if item[0] < min_x:
                            min_x = item[0]
                        if item[0] > max_x:
                            max_x = item[0]
                        if item[1] < min_y:
                            min_y = item[1]
                        if item[1] > max_y:
                            max_y = item[1]
                    mbrs[0].append([min_x,max_x,min_y,max_y])  
                    for polygons in island[1:]:
                        for item in polygons:
                            if item[0] < min_x:
                                min_x = item[0]
                            if item[0] > max_x:
                                max_x = item[0]
                            if item[1] < min_y:
                                min_y = item[1]
                            if item[1] > max_y:
                                max_y = item[1]
                        mbrs[1].append([min_x,max_x,min_y,max_y])
                    '''
                    The result is [[[x1,x2,y1,y2],[x3,x4,y3,y4]],[[x5,x6,y5,y6],[x7,x8,y7,y8]]]
                    mbrs[0]--> for all the main polygons
                    mbrs[1]--> for all of holes in polygons
                    '''
        self.MBR = mbrs
        # read the three keys in dictionary for each feature
        self.geotype = features['type']
        self.geometry = features['geometry']
        self.properties = features['properties']
    
    # to judge a point(x,y) is in a rectangular(x1,x2,y1,y2) or not
    def inRect (self, x,y, rect):
        x1,x2,y1,y2 = rect
        if x<x2 and x>x1 and y<y2 and y>y1:
            return True
        else:
            return False
    # to judge a point(x,y) is in the MBR or not    
    def inMBR (self, x,y):
        if self.geometry['type'] == 'Polygon':
            mainP = self.inRect(x,y,self.MBR[0][0])
            if mainP == False:
                return False
            for i in range(len(self.MBR[1])):
                holeP = self.inRect(x,y,self.MBR[1][i])
                if holeP == True:
                    return False
            return True
        
        if self.geometry['type'] == 'MultiPolygon':
            for i in range(len(self.MBR[0])):
                mainP = self.inRect(x,y,self.MBR[0][i])
                if mainP == False:
                    return False
            for i in range(len(self.MBR[1])):
                holeP = self.inRect(x,y,self.MBR[1][i])
                if holeP == True:
                    return False
            return True

In [30]:
import json
f = "countries.geojson"
with open(f, 'r') as infile:
    g = json.load(infile)                        
    
# Convert all the GeoJSON features to Feature objects
features = []
for i in g["features"]:
    features.append(Feature(i)) 

In [31]:
from distance import to_float
from distance import get_attribute
from distance import read_csv

data = read_csv("data.csv")
lat = to_float(get_attribute(data,'latitude'))
lon = to_float(get_attribute(data,'longitude'))
ids = to_float(get_attribute(data,'longitude'))
points = list(zip(ids,lat,lon))

In [32]:
#import pyproj
#wgs84=pyproj.Proj("+init=EPSG:4326")
for country in features:
    AinR = []
    for i,airp in enumerate(points):
        #x,y = wgs84(airp[1],airp[2])
        if country.inMBR(airp[1],airp[2]) == True:
            # For each feature add a new attribute that lists the airports contained in its MBR.
            AinR.append(data[i])
    print(len(AinR))
    country.airList = AinR

0
10
0
0
2
0
0
0
0
0
0
0
0
0
0
0
2
0
0
1
7
15
0
2
1
0
3
0
0
37
0
0
0
0
0
0
0
1
7
0
1
0
0
0
0
19
0
0
6
0
0
0
0
0
0
0
0
0
0
0
3
0
8
0
0
0
89
0
0
0
0
0
0
231
0
0
0
0
0
0
0
0
7
0
8
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
31
0
3
0
0
0
10
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
17
0
0
0
0
0
0
7
0
0
53
0
4
0
0
0
0
0
56
1
0
1
0
0
0
0
0
0
0
0
9
0
23
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
7
0
0
0
0
0
0
0
13
0
0
4
0
0
0
22
7
0
0
0
0
0
0
0
0
82
448
0
3
0
0
0
2
0
0
0
0
20
0
6
3
0
0
0
0
0
0
0
0
0
0
0
6
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
20
8


In [23]:
# If I want to check the airport list in 
print("The country is %s."%(features[1].properties["ADMIN"]))
print(features[1].geometry['type'])
print(features[1].MBR)
features[1].airList

The country is Afghanistan.
Polygon
[[[60.48677779100012, 74.89230676300014, 29.38660532600005, 38.473673402000145]], []]


[['426',
  'Immola',
  'Immola',
  'Finland',
  '',
  'EFIM',
  '61.249172',
  '28.903711',
  '338',
  '3',
  'E'],
 ['428',
  'Ivalo',
  'Ivalo',
  'Finland',
  'IVL',
  'EFIV',
  '68.607269',
  '27.405328',
  '481',
  '2',
  'E'],
 ['636',
  'Flesland',
  'Bergen',
  'Norway',
  'BGO',
  'ENBR',
  '60.293386',
  '5.218142',
  '170',
  '1',
  'E'],
 ['648',
  'Kjeller',
  'Kjeller',
  'Norway',
  '',
  'ENKJ',
  '59.969336',
  '11.036089',
  '354',
  '1',
  'E'],
 ['2948',
  'Pulkovo',
  'St. Petersburg',
  'Russia',
  'LED',
  'ULLI',
  '59.800292',
  '30.262503',
  '78',
  '4',
  'N'],
 ['4327',
  'Airport',
  'Mehamn',
  'Norway',
  'MEH',
  'ENMR',
  '71.029722',
  '27.826667',
  '41',
  '1',
  'E'],
 ['5590',
  'VÃ¦rÃ¸y Heliport',
  'VÃ¦rÃ¸y',
  'Norway',
  'VRY',
  'ENVR',
  '67.6667',
  '12.6833',
  '36',
  '1',
  'E'],
 ['6109',
  'Uzhhorod International Airport',
  'Uzhgorod',
  'Ukraine',
  'UDJ',
  'UKLU',
  '48.634278',
  '22.263356',
  '383',
  '2',
  'E'],
 ['6113',
  'K